# Chapter 8: Metaclasses and Attributes
Follow the rule of least surprise

## Item 58: Use Plain Attributes Instead of Setter and Getter Methods
Java practitioners, take note!

In [2]:
class Jimmy:
    def __init__(self, fios_installed: bool):
        self.fios_installed = fios_installed

    def get_fios_installed(self):
        return self.fios_installed

    def set_fios_installed(self, setto: bool):
        self.fios_installed = setto

    # etc etc etc

In [3]:
jimbo = Jimmy(False)
jimbo.set_fios_installed(True)
jimbo.get_fios_installed()

True

In [4]:
# But for simple cases like this, getter/setters aren't needed:
class PythonicJimmy:
    def __init__(self, fios_installed: bool):
        self.fios_installed = fios_installed

pimbo = PythonicJimmy(False)
pimbo.fios_installed = True
pimbo.fios_installed

True

In [5]:
# If we do need a more complicated interface for interacting with an attribute, use @property
class Jimmy:
    def __init__(self, spongebob_refs: int):
        self.sb_refs = spongebob_refs
        self.time_til_patrick = 10

    @property
    def spongebob_references(self):
        return self.sb_refs

    @spongebob_references.setter
    def spongebob_references(self, references: int):
        if (self.time_til_patrick > 0):
            self.sb_refs = references
            self.time_til_patrick -= 1
        else:
            raise ValueError("Jimmy is now doing a Patrick impression")

In [8]:
jimbo2 = Jimmy(0)
for x in range(0, 9):
    jimbo2.spongebob_references = x

In [21]:
# This also lets you do type checking at runtime
class Jimmy:
    def __init__(self, spongebob_refs: int):
        self.sb_refs = spongebob_refs
        self.time_til_patrick = 10

    @property
    def spongebob_references(self):
        return self.sb_refs

    @spongebob_references.setter
    def spongebob_references(self, references: int):
        if not isinstance(references, int):
            raise ValueError("NO")
        if (self.time_til_patrick > 0):
            self.sb_refs = references
            self.time_til_patrick -= 1
        else:
            raise ValueError("Jimmy is now doing a Patrick impression")

In [22]:
jimbo = Jimmy(0)
jimbo.spongebob_references = 0.5

# NOTE: you could also enforce immutability this way: raise an exception after a `hasattr` check

ValueError: NO

### Remember
- Define new class interfaces using simple public attributes and avoid defining setter and getter methods.
- Use @property to define special behavior when attributes are accessed on your objects.
- Follow the rule of least surprise and avoid odd side effects in your @property methods.
- Ensure that @property methods are fast; for slow or complex work—especially involving I/O or causing side effects—use normal methods instead.

## Item 59: Consider `@property` Instead of Refactoring Attributes

In [1]:
from datetime import datetime, timedelta

class Bucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.quota = 0

    def __repr__(self):
        return f"Bucket(quota={self.quota})"

In [2]:
def fill(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount

def deduct(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        return False  # Bucket hasn't been filled this period
    if bucket.quota - amount < 0:
        return False  # Bucket was filled, but not enough
    bucket.quota -= amount
    return True       # Bucket had enough, quota consumed

In [3]:
bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

Bucket(quota=100)


In [4]:
if deduct(bucket, 99):
    print("Had 99 quota")
else:
    print("Not enough for 99 quota")

print(bucket)

Had 99 quota
Bucket(quota=1)


In [5]:
if deduct(bucket, 3):
    print("Had 3 quota")
else:
    print("Not enough for 3 quota")

print(bucket)


# Problem here, is we never really know if we're blocked because we ran out of quota, or because we never had quota (during this time period)

Not enough for 3 quota
Bucket(quota=1)


In [7]:
class NewBucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0 # we add a max_quota to track quota issued during the period
        self.quota_consumed = 0 # and this one is, well, how much we used dawg (dawg)

    def __repr__(self):
        return (
            f"NewBucket(max_quota={self.max_quota}, "
            f"quota_consumed={self.quota_consumed})"
        )

    # And NOW we use property so this new bucket has a compatible interface!
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed

    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            # Quota being reset for a new period
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # Quota being filled during the period
            self.max_quota = amount + self.quota_consumed
        else:
            # Quota being consumed during the period
            self.quota_consumed = delta

In [8]:
bucket = NewBucket(60)
print("Initial", bucket)
fill(bucket, 100)
print("Filled", bucket)

if deduct(bucket, 99):
    print("Had 99 quota")
else:
    print("Not enough for 99 quota")

print("Now", bucket)

if deduct(bucket, 3):
    print("Had 3 quota")
else:
    print("Not enough for 3 quota")

print("Still", bucket)


Initial NewBucket(max_quota=0, quota_consumed=0)
Filled NewBucket(max_quota=100, quota_consumed=0)
Had 99 quota
Now NewBucket(max_quota=100, quota_consumed=99)
Not enough for 3 quota
Still NewBucket(max_quota=100, quota_consumed=99)


The magic of `@property` is it lets you make incremental progress towards a better data model.

> @property is a tool to help you address problems you’ll come across in real-world code. Don’t overuse it. When you find yourself repeatedly extending @property methods, it’s probably time to refactor your class instead of further paving over your code’s poor design.

# Item 60: Use Descriptors for Reusable `@property` Methods